# IMDA Dataset processing

In [1]:
import os
import re
import pandas as pd

## load results

In [2]:
rule_based_result_df = pd.read_excel("../data/processed/IMDA_Sentiment_Score--0803.xlsx")
rule_based_result_df.head()

,Unnamed: 0.1,Unnamed: 0,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment,Vader,Tblob,Stanza,Manual,GPT 0804,processed_text
0,0,0,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,0.00,5.35,hi this is A B C bank how can I help you,0.4019,0.00000,0.0,0.0,0.5,hi this be A B C bank how can I help you
1,1,1,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,17.15,23.55,ya our bank do give out ya our bank does give ...,0.0000,0.00000,0.0,NaN,0.5,ya our bank do give out ya our bank do give ou...
2,2,2,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,23.28,27.41,ya probably I can give you some information on...,0.0000,0.00000,0.0,NaN,0.5,ya probably I can give you some information on...
3,3,3,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,27.41,31.26,first okay I'm actually having a H_D_B resale ...,0.2263,0.18125,0.0,NaN,0.5,first okay I'm actually have a H_D_B resale flat
4,4,4,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,31.26,38.52,four room flat that I bought back in two thous...,0.0000,-0.01250,-1.0,NaN,0.5,four room flat that I buy back in thousand six...


In [3]:
rule_based_result_df = rule_based_result_df.drop(columns=["Unnamed: 0.1","Unnamed: 0"])

In [4]:
gpt_based_result_df = pd.read_csv("../data/processed/temp_test_6Aug.csv")
gpt_based_result_df.head()

,Unnamed: 0,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment,sentiment_score_GPT
0,58,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,0.00,6.60,hi good morning,0.0
1,59,app_0683_0013_phnd_cc-ins.TextGrid,683,13,agent,insurance,1.93,5.53,hi this is A B C insurance company how can I h...,0.0
2,60,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,6.60,9.63,hi morning my name is john I'm calling in to (uh),0.0
3,61,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,32.07,39.33,(mm) so does the (ppb) does your insurance has...,0.0
4,62,app_0683_0013_phnd_cc-ins.TextGrid,683,13,agent,insurance,40.13,46.78,ya we can definitely help you (uh) convert it ...,1.0


In [5]:
# gpt_based_result_df[['session_id','speaker_id','dialog_type','x_min','sentiment_score_GPT']].head()

,session_id,speaker_id,dialog_type,x_min,sentiment_score_GPT
0,683,13,bank,0.00,0.5
1,683,13,bank,17.15,0.5
2,683,4366,bank,23.28,0.5
3,683,4366,bank,27.41,0.5
4,683,4366,bank,31.26,0.5


In [6]:
# rule_based_result_df[['session_id','speaker_id','dialog_type','x_min','Vader','Tblob','Stanza','Manual']].head()

,session_id,speaker_id,dialog_type,x_min,Vader,Tblob,Stanza,Manual
0,683,13,bank,0.00,0.4019,0.00000,0.0,0.0
1,683,13,bank,17.15,0.0000,0.00000,0.0,NaN
2,683,4366,bank,23.28,0.0000,0.00000,0.0,NaN
3,683,4366,bank,27.41,0.2263,0.18125,0.0,NaN
4,683,4366,bank,31.26,0.0000,-0.01250,-1.0,NaN


In [5]:
merged_df = pd.merge(rule_based_result_df, gpt_based_result_df, 
                     on=['file_name','session_id', 'speaker_id','speaker_type', 'dialog_type', 'x_min','x_max'])
merged_df.head()

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,cleaned_text_for_sentiment_x,Vader,Tblob,Stanza,Manual,GPT 0804,processed_text,Unnamed: 0,cleaned_text_for_sentiment_y,sentiment_score_GPT
0,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,0.00,6.60,hi good morning,0.4404,0.7,1.0,0.0,0.5,hi good morning,58,hi good morning,0.0
1,app_0683_0013_phnd_cc-ins.TextGrid,683,13,agent,insurance,1.93,5.53,hi this is A B C insurance company how can I h...,0.4019,0.0,0.0,NaN,0.5,hi this be A B C insurance company how can I h...,59,hi this is A B C insurance company how can I h...,0.0
2,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,6.60,9.63,hi morning my name is john I'm calling in to,0.0000,0.0,0.0,NaN,0.5,hi morning my name be john I'm call in to,60,hi morning my name is john I'm calling in to (uh),0.0
3,app_0683_4366_phnd_cc-ins.TextGrid,683,4366,client,insurance,32.07,39.33,so does the does your insurance has helped to ...,0.4019,0.0,-1.0,NaN,0.5,so do the do your insurance have help to facil...,61,(mm) so does the (ppb) does your insurance has...,0.0
4,app_0683_0013_phnd_cc-ins.TextGrid,683,13,agent,insurance,40.13,46.78,ya we can definitely help you convert it from ...,0.6597,0.0,0.0,NaN,0.7,ya we can definitely help you convert it from ...,62,ya we can definitely help you (uh) convert it ...,1.0


In [6]:
merged_df = merged_df.drop(columns=['GPT 0804','cleaned_text_for_sentiment_y'])	

In [7]:
merged_df.to_excel("../data/processed/IMDA_Sentiment_Score_0803_merged_6Aug_test.xlsx",index=False)